In [9]:
import torch
import torch.onnx

from maraboupy import Marabou
import numpy as np

/home/saimadhavang/.local/lib/python3.10/site-packages/maraboupy/Marabou.py:27: UserWarning: Tensorflow parser is unavailable because tensorflow package is not installed
  warnings.warn("Tensorflow parser is unavailable because tensorflow package is not installed")


In [10]:
net = Marabou.read_onnx("./mnist-net_256x4.onnx")

In [11]:
ds =torch.load("./dataset/test.pt")

In [12]:
example = ds[0]

In [13]:
example = example/255

In [14]:
batch = []
for e in range(example.shape[0]):
    batch.append(example[e].reshape((1, 784)))

In [40]:
batch[0].shape

torch.Size([1, 784])

In [41]:
cnt = 0

In [35]:
for i in range(len(batch)):
    res = net.evaluateWithoutMarabou(batch[i].numpy())
    pred = np.argmax(res)
    gt = ds[1][i]
    if pred == gt:
        cnt += 1
print(f"Accuracy: {cnt/len(batch)}")

Accuracy: 0.9764


In [4]:
from torchvision import datasets, transforms

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

test_set = datasets.MNIST('./data', train=False,
                          transform=transform)

In [16]:
cnt = 0

In [18]:
for i in range(len(test_set)):
    example = test_set[i][0].numpy().reshape((1, 784))
    res = net.evaluateWithoutMarabou(example)
    pred = np.argmax(res)
    gt = test_set[i][1]
    if pred == gt:
        cnt += 1
print(f"Accuracy: {cnt/len(test_set)}")

Accuracy: 0.8539


In [34]:
train_set = datasets.MNIST('./data', train=True,
                          transform=transform)

In [35]:
normalized_train = torch.empty((60000, 28, 28))

In [36]:
for i in range(60000):
    normalized_train[i] = train_set[i][0]

In [37]:
torch.save(normalized_train, "./dataset/normalized_train.pt")

In [38]:
nt_targets = torch.empty(60000)

In [39]:
for i in range(60000):
    nt_targets[i] = train_set[i][1]

In [40]:
torch.save(nt_targets, "./dataset/nt_targets.pt")